In [1]:
from utils import *
from logic import *
from notebook import psource

# crime KB 입력

In [2]:
clauses = []
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))
clauses.append(expr("Enemy(Nono, America)"))
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))
clauses.append(expr("American(West)"))
clauses.append(expr("Missile(x) ==> Weapon(x)"))
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))
crime_kb = FolKB(clauses)

# Backward chaining Algorithm

In [3]:
def fol_bc_ask(kb, query):
    return fol_bc_or(kb, query, {})

#### OR

In [4]:
def fol_bc_or(KB, goal, theta):
    for rule in KB.fetch_rules_for_goal(goal):
        lhs, rhs = parse_definite_clause(standardize_variables(rule))
        for theta1 in fol_bc_and(KB, lhs, unify(rhs, goal, theta)):
            yield theta1

### And

In [5]:
def fol_bc_and(KB, goals, theta):
    if theta is None:
        pass
    elif not goals:
        yield theta
    else:
        first, rest = goals[0], goals[1:]
        for theta1 in fol_bc_or(KB, subst(theta, first), theta):
            for theta2 in fol_bc_and(KB, rest, theta1):
                yield theta2

# Result

In [6]:
crime_kb.ask(expr('Hostile(x)'))

{v_5: Nono, x: Nono}

In [7]:
crime_kb.ask(expr('Criminal(x)'))

{v_9: West, x: West, v_20: M1, v_10: M1, v_27: M1, v_11: Nono, v_39: Nono}

In [8]:
crime_kb.ask(expr('Criminal(West)'))

{v_43: West, v_54: M1, v_44: M1, v_61: M1, v_45: Nono, v_73: Nono}

In [9]:
crime_kb.ask(expr('American(x)'))

{x: West}

In [10]:
crime_kb.ask(expr('Sells(x, y, z)'))

{v_84: M1, x: West, z: Nono, y: M1}

In [11]:
crime_kb.ask(expr('Weapon(y)'))

{v_95: M1, y: M1}

In [12]:
%%timeit -n 100
fol_fc_ask(crime_kb, expr('Criminal(x)'))

8.41 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit -n 100
crime_kb.ask(expr('Criminal(x)'))

1.51 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Practice

### 산업공학과 대학원 석사과정 졸업요건('21. 3월 개정)
##### 실습을 위하여 상당 부분 요약 또는 생략하였습니다.
1. 분야별 과목 수강: OMS, DS, TM 분야 모두를 수강해야 한다.
2. 영어시험 합격: TOEFL 또는 TOEIC 또는 TEPS을 합격해야 한다.
3. 논문을 발표해야 한다.

Graduate(x): x can get a master's degree\
Student(x): x is a student\
Subject(x, y): x passed all subjects y\
English(x, y): x acquired a english certification y\
Paper(x, y): x released a paper on the subject y

Student(a) & Subject(a, x) & English(a, y) & Paper(a, z) => Graduate(a)

##### case: 학생A는 분야별 과목을 모두 수강하였으며 영어시험은 TOEIC으로 합격했고 DS분야로 논문 발표를 마쳤다.

In [14]:
clauses = []

In [15]:
clauses.append(expr('Student(a) & English(a, x) & Subject(a, o, d, t) & Paper(a, y) ==> Graduate(a)'))

In [16]:
clauses.append(expr('Student(Std_A)'))

In [17]:
clauses.append(expr('(OMS(x) & DS(y) & TM(z)) ==> Subject(a, x, y, z)'))
clauses.append(expr('OMS_pass(O)'))
clauses.append(expr('DS_pass(D)'))
clauses.append(expr('TM_pass(T)'))
clauses.append(expr('OMS_pass(x) ==> OMS(x)'))
clauses.append(expr('DS_pass(x) ==> DS(x)'))
clauses.append(expr('DS(x) ==> Paper(a, x)'))
clauses.append(expr('TM_pass(x) ==> TM(x)'))

In [18]:
clauses.append(expr('TOEIC(P)'))

In [19]:
clauses.append(expr('TOEIC(x) ==> English(a, x)'))

In [20]:
Gr_kb = FolKB(clauses)

##### Fol Backward Chaining Algorithm result

In [21]:
Gr_kb.ask(expr('Graduate(a)'))

{v_23899: Std_A,
 a: Std_A,
 v_23927: Std_A,
 v_23926: P,
 v_23900: P,
 v_23952: Std_A,
 v_23949: O,
 v_23950: D,
 v_23951: T,
 v_23963: O,
 v_23901: O,
 v_23985: D,
 v_23902: D,
 v_24010: T,
 v_23903: T,
 v_24034: Std_A,
 v_24033: D,
 v_24046: D,
 v_23904: D}

In [22]:
from notebook import Canvas_fol_bc_ask
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', Gr_kb, expr('Graduate(a)'))